In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, execute
from qiskit_ibm_provider import IBMProvider
import qiskit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from QuAwesome import DataManager
import qutip
from qutip import Qobj
from qutip import ptrace

/tmp/ipykernel_73607/4165708055.py:3: DeprecationWarning: The package qiskit_ibm_provider is being deprecated. Please see https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime to get instructions on how to migrate to qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from qiskit_ibm_provider import IBMProvider


In [2]:
# set up the ibmq account 

# only exceute once , it will load automatically afterward
IBMProvider.save_account(token="5e323c289cb41e03e565a0c882fcf8a92d44d7ff175b755a59b982629216c24369583d3311ae92751dc459f5e257f9ee4e2c26e03c738db142ecb296564a87b9",overwrite=True)
provider = IBMProvider()

In [3]:

import collections
from qutip import Qobj

cvt = {'0' : 1 , '1' : -1}
def get_exp(k , iid):
    rs = 1
    for idx ,i in enumerate(k[::-1]) :
        if idx in iid :
            rs = rs
        else :
            rs = rs * cvt[i]
    return rs
def calculate_exp(source , iid):
    rs = 0
    shots = sum([v for v in source.values()])
    for k,v in source.items() :
        rs += get_exp(k , iid) * (v/shots)
    return rs
def Tomography(raw_counts) :
    # X Y Z
    #print("======")
    iid = Qobj(np.array([[1, 0], [0, 1]])) # I
    pauli = [
        Qobj(np.array([[0, 1], [1, 0]])) , # X
        Qobj(np.array([[0, -1j], [1j, 0]])) , # Y
        Qobj(np.array([[1, 0], [0, -1]])) , # Z
        iid # I
    ]
    name = ['X' ,'Y' ,'Z' , "I"]
   
    rho = qutip.tensor(iid , iid ,iid) # I*I*I
    ctrl = [
        [0] ,
        [0] ,
        [0,1]
    ]
    for ql0 in range(4) :
        q0 = ql0
        if ql0 ==  3 :
            q0 = 2
        for ql1 in range(4) :
            q1 = ql1
            if ql1 == 3 :
                q1 = 2
            for ql2 in range(4):
                q2 = ql2
                if ql2 == 3 :
                    q2 = 2
                pauli_mt  = qutip.tensor(pauli[ql0] , pauli[ql1] , pauli[ql2])
                tg = dict(raw_counts[q0][q1][q2])
                qiid = [idx for idx ,i in enumerate([ql0 ,ql1 ,ql2]) if i == 3 ]
                v = calculate_exp(tg , qiid)
                #print(name[ql0] +name[ql1] + name[ql2] , v)
                rho += pauli_mt * v
                
    rho -= qutip.tensor(iid , iid ,iid)
    #print(rho.tr())
    return rho*0.125

In [78]:
iid = Qobj(np.array([[1, 0], [0, 1]])) # I
EEE = Qobj(np.array([[0, 0], [0, 1]])) # I
GGG = Qobj(np.array([[1, 0], [0, 0]])) # I
aaa = qutip.tensor(EEE,GGG) # I*I*I

In [79]:
aaa

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

# Quantum Circuit

In [8]:
# define the physical qubit you wanna use
# choose the qubit on the ibmq page
# q = [9,62,65] 
# qMax = max(q) + 1
qMax = 3
# register the qubit and classical bit
Q = QuantumRegister(qMax, name='q')
# C = ClassicalRegister(2, name='bell')

In [9]:
# Generate 3-qubit GHZ state
qc = qiskit.QuantumCircuit(Q)
qc.h(0)
qc.cx(0, (1))

qc.barrier(0,1,2)
qc.cu(np.pi/2,0,0,0, 1, 2)

qc.cx(2,1) 

In [10]:
print(qc)

     ┌───┐      ░                      
q_0: ┤ H ├──■───░──────────────────────
     └───┘┌─┴─┐ ░                 ┌───┐
q_1: ─────┤ X ├─░────────■────────┤ X ├
          └───┘ ░ ┌──────┴───────┐└─┬─┘
q_2: ───────────░─┤ U(π/2,0,0,0) ├──■──
                ░ └──────────────┘     


In [97]:
# q=[18,19,20]
q=[0,1,2]
qMax = max(q) + 1
Q = QuantumRegister(qMax, name='q')
c_bell = ClassicalRegister(3, name='tomography')

qcs = []
for ql0 in range(3) :
    for ql1 in range(3) :
        for ql2 in range(3):
            qc = QuantumCircuit(Q, c_bell)

            qc.h(Q[q[0]])
            qc.cx(Q[q[0]], (Q[q[1]]))
            # qc.barrier(0,1,2)

            qc.cu(np.pi/2,0,0,0,Q[q[1]], Q[q[2]])
            # qc.cu(0,0,0,0,Q[q[1]], Q[q[2]])
            qc.cx(Q[q[2]], (Q[q[1]]))

            

            qc.barrier(range(qMax))
            # tomography
            if ql0 == 0 :     # measure x
                qc.h(Q[q[0]])
            if ql0 == 1 :     # measure y
                qc.sdg(Q[q[0]])
                qc.h(Q[q[0]])
                
            if ql1 == 0 :     # measure x
                qc.h(Q[q[1]])
            if ql1 == 1 :     # measure y
                qc.sdg(Q[q[1]])
                qc.h(Q[q[1]])
                
            if ql2 == 0 :     # measure x
                qc.h(Q[q[2]])
            if ql2 == 1 :     # measure y
                qc.sdg(Q[q[2]])
                qc.h(Q[q[2]])
            qc.measure([Q[q[0]], Q[q[1]] , Q[q[2]]], [c_bell[0], c_bell[1] , c_bell[2]])
            # transpile qc
            qcs.append(qc)

In [98]:
print(qcs[1])

              ┌───┐                           ░  ┌───┐      ┌─┐      
         q_0: ┤ H ├──■────────────────────────░──┤ H ├──────┤M├──────
              └───┘┌─┴─┐                ┌───┐ ░  ├───┤      └╥┘┌─┐   
         q_1: ─────┤ X ├───────■────────┤ X ├─░──┤ H ├───────╫─┤M├───
                   └───┘┌──────┴───────┐└─┬─┘ ░ ┌┴───┴┐┌───┐ ║ └╥┘┌─┐
         q_2: ──────────┤ U(π/2,0,0,0) ├──■───░─┤ Sdg ├┤ H ├─╫──╫─┤M├
                        └──────────────┘      ░ └─────┘└───┘ ║  ║ └╥┘
tomography: 3/═══════════════════════════════════════════════╩══╩══╩═
                                                             0  1  2 


# Local Simulation

In [99]:

# Construct an ideal simulator
aersim = AerSimulator(shots=10000)

# Perform an ideal simulation
result_ideal = aersim.run(qcs).result()
# counts_ideal = result_ideal.get_state()
# print('Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

# Counts(noise): {'101': 16, '110': 48, '100': 7, '001': 31, '010': 7, '000': 464, '011': 15, '111': 436}

In [100]:
t = result_ideal.get_counts()
shape=(3,3,3)
count_list = np.reshape(np.array(t) ,shape)

In [101]:
ρ=Tomography(count_list)
ptrace(ρ,(0,1))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 4.9800e-01+0.j      -7.3500e-03+0.00215j  3.5500e-03+0.0011j
   3.4995e-01-0.00245j]
 [-7.3500e-03-0.00215j  0.0000e+00+0.j      -2.3500e-03+0.00085j
  -1.1500e-03-0.0007j ]
 [ 3.5500e-03-0.0011j  -2.3500e-03-0.00085j  2.4510e-01+0.j
  -1.5000e-04-0.00235j]
 [ 3.4995e-01+0.00245j -1.1500e-03+0.0007j  -1.5000e-04+0.00235j
   2.5690e-01+0.j     ]]

In [52]:
0.354*2

0.708

# Real device

In [53]:
provider.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibm_sherbrooke')>]

In [54]:
# select the machine ; quantum device or local simulator
# check out homepage of ibmq for accessible device
machine = 'ibm_osaka'
backend = provider.get_backend(machine)

## Execute

In [55]:
shot=10000
data = {
    'backend': None,
    'job_id' : None,
    'q_list' : [],
    'shots'  : shot,
    'Robust' : None,
    'Signal' : None,
    'assemb' : None,
    'counts' : [],
    'Memory' : []
}
DM = DataManager(data)

In [56]:
job = execute(qcs, backend=backend, shots=shot, optimization_level=0, memory=True)

In [57]:

DM['job_id'] = job.job_id()
DM['job_id']

'csfxpmabm6a000897jn0'

In [58]:
FILENAME = "Amplitude"
DM.save(FILENAME, False)

Save data to 'Amplitude.json' (success)


In [61]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

# Result

In [75]:
result = job.result()
DM['counts'] = result.get_counts()


In [76]:
DM['counts'][3]

{'100': 561,
 '101': 1939,
 '110': 508,
 '001': 478,
 '111': 1931,
 '010': 1999,
 '011': 567,
 '000': 2017}

In [77]:
shape=(3,3,3)
count_list_real = np.reshape(np.array(DM['counts']) ,shape)

ρ=Tomography(count_list_real)
ptrace(ρ,(0,1))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 4.736e-01+0.j     -1.250e-02+0.0497j  1.490e-02+0.0288j
   2.841e-01-0.003j ]
 [-1.250e-02-0.0497j  3.400e-02+0.j      7.800e-03-0.0087j
   4.000e-04-0.0414j]
 [ 1.490e-02-0.0288j  7.800e-03+0.0087j  2.742e-01+0.j
   1.120e-02-0.0552j]
 [ 2.841e-01+0.003j   4.000e-04+0.0414j  1.120e-02+0.0552j
   2.182e-01+0.j    ]]

In [80]:
ptrace(ρ,(0,1))*2

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 9.472e-01+0.j     -2.500e-02+0.0994j  2.980e-02+0.0576j
   5.682e-01-0.006j ]
 [-2.500e-02-0.0994j  6.800e-02+0.j      1.560e-02-0.0174j
   8.000e-04-0.0828j]
 [ 2.980e-02-0.0576j  1.560e-02+0.0174j  5.484e-01+0.j
   2.240e-02-0.1104j]
 [ 5.682e-01+0.006j   8.000e-04+0.0828j  2.240e-02+0.1104j
   4.364e-01+0.j    ]]